In [31]:
# Cell 1: Health Check Test
import requests
import json

def test_health_check():
    """Test the backend health endpoint"""
    try:
        response = requests.get('http://localhost:5000/api/health')
        print("Health Check Response:")
        print(json.dumps(response.json(), indent=2))
        print(f"Status Code: {response.status_code}")
        return response.json()
    except Exception as e:
        print(f"Health check failed: {e}")
        return None

test_health_check()

Health Check Response:
{
  "services": {
    "enrichment_service": "available",
    "mock_service": "available"
  },
  "status": "healthy"
}
Status Code: 200


{'services': {'enrichment_service': 'available', 'mock_service': 'available'},
 'status': 'healthy'}

In [32]:
# Cell 2: Lead Search Test
import requests
import json

def test_lead_search():
    """Test the lead search functionality"""
    search_payload = {
        "query": "software development Chicago",
        "limit": 10,
        "location": "Chicago, IL",
        "use_mock": True
    }
    
    try:
        response = requests.post(
            'http://localhost:5000/api/leads/search',
            headers={'Content-Type': 'application/json'},
            json=search_payload
        )
        
        print("Lead Search Response:")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"Success: {data['success']}")
            print(f"Found {data['count']} leads")
            print(f"Query: {data['query']}")
            print(f"Mode: {data['mode']}")
            
            for i, lead in enumerate(data['leads'][:3]):
                print(f"\nLead {i+1}:")
                print(f"  Company: {lead['company']}")
                print(f"  Industry: {lead['industry']}")
                print(f"  Website: {lead['website']}")
                print(f"  Phone: {lead.get('phone', 'N/A')}")
                
            return data['leads']
        else:
            print(f"Error: {response.text}")
            return None
            
    except Exception as e:
        print(f"Search failed: {e}")
        return None

leads = test_lead_search()

Lead Search Response:
Status Code: 200
Success: True
Found 10 leads
Query: software development Chicago
Mode: mock

Lead 1:
  Company: CloudBridge Corp 1
  Industry: Healthcare
  Website: https://cloudbridgecorp.com
  Phone: +1-555-522-9325

Lead 2:
  Company: CloudBridge Corp 2
  Industry: Technology
  Website: https://cloudbridgecorp.com
  Phone: +1-555-960-3193

Lead 3:
  Company: TechFlow Solutions 3
  Industry: SaaS
  Website: https://techflowsolutions.com
  Phone: +1-555-849-9838


In [33]:
# Cell 3: Lead Enrichment Test
import requests
import json

def test_lead_enrichment():
    """Test the lead enrichment functionality"""
    sample_leads = [
        {
            "company": "TechCorp Solutions",
            "industry": "Software Development",
            "address": "123 Tech Street, Chicago, IL",
            "phone": "555-0123",
            "website": "https://techcorp.com",
            "source": "mock",
            "created_at": "2025-01-11T12:00:00Z"
        },
        {
            "company": "DataFlow Inc",
            "industry": "Data Analytics",
            "address": "456 Data Ave, San Francisco, CA",
            "phone": "555-0456",
            "website": "https://dataflow.com",
            "source": "mock",
            "created_at": "2025-01-11T12:00:00Z"
        }
    ]
    
    enrichment_payload = {
        "leads": sample_leads,
        "use_mock": True,
        "user_preferences": {
            "target_industries": ["Software Development", "SaaS"],
            "min_employees": 10,
            "max_employees": 500,
            "business_type_preference": "B2B"
        }
    }
    
    try:
        response = requests.post(
            'http://localhost:5000/api/leads/enrich',
            headers={'Content-Type': 'application/json'},
            json=enrichment_payload
        )
        
        print("Lead Enrichment Response:")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"Success: {data['success']}")
            print(f"Enriched {data['count']} leads")
            
            for i, lead in enumerate(data['enriched_leads']):
                print(f"\nEnriched Lead {i+1}:")
                print(f"  Company: {lead['company']}")
                print(f"  Industry: {lead['industry']}")
                print(f"  Employees: {lead['employees']}")
                print(f"  Revenue: {lead['revenue']}")
                print(f"  Fit Score: {lead['fit_score']}/10")
                print(f"  Activity Score: {lead['activity_score']}/10")
                print(f"  Tags: {', '.join(lead['tags'])}")
                
                if lead.get('owner_info'):
                    owner = lead['owner_info']
                    print(f"  Contact: {owner['name']} ({owner['email']})")
                    print(f"  Title: {owner['title']}")
                    print(f"  LinkedIn: {owner['linkedin_url']}")
                
            return data['enriched_leads']
        else:
            print(f"Error: {response.text}")
            return None
            
    except Exception as e:
        print(f"Enrichment failed: {e}")
        return None

enriched_leads = test_lead_enrichment()

Lead Enrichment Response:
Status Code: 200
Success: True
Enriched 2 leads

Enriched Lead 1:
  Company: TechCorp Solutions
  Industry: Software Development
  Employees: 384
  Revenue: $1M-$5M
  Fit Score: 8.4/10
  Activity Score: 7.9/10
  Tags: enriched, high-priority, b2b
  Contact: Contact at TechCorp Solutions (contact@techcorpsolutions.com)
  Title: Business Development Manager
  LinkedIn: https://linkedin.com/in/contact-techcorp-solutions

Enriched Lead 2:
  Company: DataFlow Inc
  Industry: Data Analytics
  Employees: 224
  Revenue: $1M-$5M
  Fit Score: 7.2/10
  Activity Score: 9.5/10
  Tags: enriched, high-priority, b2b
  Contact: Contact at DataFlow Inc (contact@dataflowinc.com)
  Title: Business Development Manager
  LinkedIn: https://linkedin.com/in/contact-dataflow-inc


In [34]:
# Cell 4: Email System Test (Python requests)
import requests
import json

def test_email_system():
    """Test the email system with sample leads"""
    
    # Sample enriched leads for email demo
    sample_leads = [
        {
            "id": "demo-1",
            "company": "InnovateTech Solutions",
            "industry": "Software Development",
            "website": "https://innovatetech.com",
            "phone": "555-0123",
            "address": "123 Innovation Drive, San Francisco, CA",
            "employees": 150,
            "revenue": "$5M-$25M",
            "owner_name": "Sarah Johnson",
            "owner_email": "sarah.johnson@innovatetech.com",
            "fit_score": 8.7,
            "tags": ["enriched", "high-priority", "b2b"],
            "is_enriched": True
        },
        {
            "id": "demo-2",
            "company": "DataFlow Analytics",
            "industry": "Data Analytics",
            "website": "https://dataflow.com",
            "phone": "555-0456",
            "address": "456 Analytics Ave, Chicago, IL",
            "employees": 75,
            "revenue": "$1M-$5M",
            "owner_name": "Michael Chen",
            "owner_email": "m.chen@dataflow.com",
            "fit_score": 7.9,
            "tags": ["enriched", "good-match", "b2b"],
            "is_enriched": True
        }
    ]
    
    email_payload = {
        "email": "jesayadavidgnp@gmail.com",  
        "leads": sample_leads
    }
    
    try:
        response = requests.post(
            'http://localhost:3000/api/email-summary',
            headers={'Content-Type': 'application/json'},
            json=email_payload
        )
        
        print("Email System Response:")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print(f"Success: {data.get('success', False)}")
            if data.get('messageId'):
                print(f"Message ID: {data['messageId']}")
            print(f"Email includes {len(sample_leads)} leads")
            print(f"Top leads: {len([l for l in sample_leads if l.get('fit_score', 0) >= 8])}")
            print(f"Enriched leads: {len([l for l in sample_leads if l.get('is_enriched')])}")
        else:
            print(f"Error: {response.text}")
            
        return response.json() if response.status_code == 200 else None
        
    except Exception as e:
        print(f"Email test failed: {e}")
        return None

email_result = test_email_system()

Email System Response:
Status Code: 200
Success: True
Message ID: 57ac051f-327d-44d8-ae10-164055cceb09
Email includes 2 leads
Top leads: 1
Enriched leads: 2


In [35]:
# Cell 5: Complete Workflow Test
import requests
import json
import time

def test_complete_workflow():
    """Test the complete SaaSquatch workflow"""
    print("=== Complete SaaSquatch Workflow Demo ===")
    
    try:
        # Step 1: Health Check
        print("\nStep 1: Checking backend health...")
        health_response = requests.get('http://localhost:5000/api/health')
        if health_response.status_code == 200:
            print("Backend is healthy")
        else:
            print("Backend health check failed")
            return None
        
        # Step 2: Search for leads
        print("\nStep 2: Searching for leads...")
        search_payload = {
            "query": "AI startups San Francisco",
            "limit": 8,
            "location": "San Francisco, CA",
            "industry": "Artificial Intelligence",
            "min_employees": 5,
            "max_employees": 200,
            "use_mock": True
        }
        
        search_response = requests.post(
            'http://localhost:5000/api/leads/search',
            headers={'Content-Type': 'application/json'},
            json=search_payload
        )
        
        if search_response.status_code != 200:
            print(f"Search failed: {search_response.text}")
            return None
            
        search_data = search_response.json()
        print(f"Found {search_data['count']} leads")
        
        # Step 3: Enrich selected leads
        if search_data['success'] and search_data['leads']:
            print("\nStep 3: Enriching leads with additional data...")
            
            # Take first 5 leads for enrichment
            leads_to_enrich = search_data['leads'][:5]
            
            enrichment_payload = {
                "leads": leads_to_enrich,
                "use_mock": True,
                "user_preferences": {
                    "target_industries": ["Artificial Intelligence", "Software"],
                    "business_type_preference": "B2B",
                    "min_fit_score": 7.0
                }
            }
            
            enrich_response = requests.post(
                'http://localhost:5000/api/leads/enrich',
                headers={'Content-Type': 'application/json'},
                json=enrichment_payload
            )
            
            if enrich_response.status_code != 200:
                print(f"Enrichment failed: {enrich_response.text}")
                return None
                
            enrich_data = enrich_response.json()
            print(f"Enriched {enrich_data['count']} leads")
            
            # Step 4: Prepare email data
            print("\nStep 4: Preparing email summary...")
            
            # Convert enriched leads to email format
            email_leads = []
            for i, lead in enumerate(enrich_data['enriched_leads']):
                email_lead = {
                    "id": f"enriched-{i}",
                    "company": lead['company'],
                    "industry": lead.get('industry', 'Unknown'),
                    "website": lead.get('website', ''),
                    "phone": lead.get('phone'),
                    "address": lead.get('address', ''),
                    "employees": lead.get('employees'),
                    "revenue": lead.get('revenue'),
                    "fit_score": lead.get('fit_score'),
                    "tags": lead.get('tags', []),
                    "is_enriched": True
                }
                
                if lead.get('owner_info'):
                    email_lead["owner_name"] = lead['owner_info'].get('name')
                    email_lead["owner_email"] = lead['owner_info'].get('email')
                    
                email_leads.append(email_lead)
            
            # Display summary
            print("Complete workflow successful!")
            print("Summary:")
            print(f"  • Searched: {search_data['count']} leads found")
            print(f"  • Enriched: {enrich_data['count']} leads with full data")
            print(f"  • High-fit leads: {len([l for l in email_leads if l.get('fit_score', 0) >= 8])}")
            print(f"  • With contacts: {len([l for l in email_leads if l.get('owner_email')])}")
            
            return {
                "searchCount": search_data['count'],
                "enrichedCount": enrich_data['count'],
                "leads": email_leads,
                "workflow_success": True
            }
        
    except Exception as e:
        print(f"❌ Complete workflow demo failed: {e}")
        return None

workflow_result = test_complete_workflow()

=== Complete SaaSquatch Workflow Demo ===

Step 1: Checking backend health...
Backend is healthy

Step 2: Searching for leads...
Found 8 leads

Step 3: Enriching leads with additional data...
Enriched 5 leads

Step 4: Preparing email summary...
Complete workflow successful!
Summary:
  • Searched: 8 leads found
  • Enriched: 5 leads with full data
  • High-fit leads: 3
  • With contacts: 5


In [36]:
# Cell 6: Start Backend Server (if needed)
import subprocess
import os
import time

def start_backend_server():
    """Start the Flask backend server"""
    try:
        # Check if backend is already running
        health_check = requests.get('http://localhost:5000/api/health', timeout=2)
        if health_check.status_code == 200:
            print("✅ Backend is already running!")
            return True
    except:
        print("Backend not running, attempting to start...")
    
    try:
        # Change to backend directory and start server
        backend_path = os.path.join(os.getcwd(), 'bePy')
        if os.path.exists(backend_path):
            print(f"Starting backend from: {backend_path}")
            # Note: This will start the server in the background
            # You might need to run this in a separate terminal
            print("Run this command in a separate terminal:")
            print(f"cd {backend_path} && python app.py")
            return True
        else:
            print(f"❌ Backend directory not found: {backend_path}")
            return False
            
    except Exception as e:
        print(f"❌ Failed to start backend: {e}")
        return False

# Uncomment to start backend if needed
# start_backend_server()

In [39]:
# Cell 7: Display Results Summary
def display_demo_summary():
    """Display a summary of all demo results"""
    print("=" * 60)
    print("SAASQUATCH DEMO SUMMARY")
    print("=" * 60)
    
    print("\nAvailable Demo Functions:")
    print("1. test_health_check() - Test backend connectivity")
    print("2. test_lead_search() - Search for leads")
    print("3. test_lead_enrichment() - Enrich lead data")
    print("4. test_email_system() - Test email functionality")
    print("5. test_complete_workflow() - Full end-to-end test")
    
    print("\nSystem Architecture:")
    print("• Backend (Flask): http://localhost:5000")
    print("• Frontend (Next.js): http://localhost:3000")
    print("• API Endpoints:")
    print("  - GET /api/health")
    print("  - POST /api/leads/search")
    print("  - POST /api/leads/enrich")
    print("  - POST /api/email-summary")
    
    print("\nTo run the complete demo:")
    print("1. Start backend: cd bePy && python app.py")
    print("2. Start frontend: cd feNext/fenext && npm run dev")
    print("3. Run: workflow_result = test_complete_workflow()")
    
    print("\nEmail System:")
    print("• Uses Next.js API route for email sending")
    print("• Formats lead data into HTML email template")
    print("• Includes fit scores and enrichment data")
    
display_demo_summary()

SAASQUATCH DEMO SUMMARY

Available Demo Functions:
1. test_health_check() - Test backend connectivity
2. test_lead_search() - Search for leads
3. test_lead_enrichment() - Enrich lead data
4. test_email_system() - Test email functionality
5. test_complete_workflow() - Full end-to-end test

System Architecture:
• Backend (Flask): http://localhost:5000
• Frontend (Next.js): http://localhost:3000
• API Endpoints:
  - GET /api/health
  - POST /api/leads/search
  - POST /api/leads/enrich
  - POST /api/email-summary

To run the complete demo:
1. Start backend: cd bePy && python app.py
2. Start frontend: cd feNext/fenext && npm run dev
3. Run: workflow_result = test_complete_workflow()

Email System:
• Uses Next.js API route for email sending
• Formats lead data into HTML email template
• Includes fit scores and enrichment data
